In [1]:
%matplotlib inline
import segpy
from segpy.reader import create_reader
from segpy.writer import write_segy

import numpy as np
import matplotlib
import matplotlib.pyplot
from PIL import Image

import os
import shutil

from collections import defaultdict, Counter
import joblib
import pickle

from scipy.stats import mode
from scipy.special import kl_div
from scipy.stats import entropy

from sklearn.preprocessing import MinMaxScaler

# %run bp_smc_example.ipynb

In [2]:
len(os.listdir('small_dataset'))

120

In [3]:
# split up stack and gather files into separate directories
for filename in os.listdir('small_dataset'):
    src = 'small_dataset/' + filename
    if 'gather' in filename:
        dst = 'gather/' + filename
        shutil.copy(src, dst)
    if 'stack' in filename:
        dst = 'stack/' + filename
        shutil.copy(src, dst)

In [4]:
# calculate standard deviation of every pixel? 
print_segy_info('gather/img_1.gather.segy')

NameError: name 'print_segy_info' is not defined

In [ ]:
print_segy_info('stack/img_12.stack.segy')

In [19]:
# make a dictionary of arrays for every realization in stack
stack_dict = {}
for filename in os.listdir('stack')[1:-1]:
    arr = np.zeros((1058, 400))

    with open(f'stack/{filename}', 'rb') as segy_in_file:
        segy_reader = create_reader(segy_in_file, endian='>')

        for trace_index in segy_reader.trace_indexes():
            data = segy_reader.trace_samples(trace_index)
            for i in range(len(data)):
                arr[trace_index, i] = data[i]
        stack_dict[filename] = arr

In [20]:
# make a dictionary of arrays for every realization in stack
gather_dict = {}
for filename in os.listdir('gather')[1:-1]:
    arr = np.zeros((1058, 39, 400))
    with open(f'gather/{filename}', 'rb') as segy_in_file:
        segy_reader = create_reader(segy_in_file, endian='>')

        count = 0
        for trace_index in segy_reader.trace_indexes():
            data = segy_reader.trace_samples(trace_index)
            for i in range(len(data)):
                arr[int(trace_index / 39), trace_index % 39, i] = data[i]
        gather_dict[filename] = arr

In [5]:
# load the two dictionaries into the directory to use later
# joblib.dump(stack_dict, 'stack_dict')
stack_dict = joblib.load(open('stack_dict', 'rb'))

# joblib.dump(gather_dict, 'gather_dict')
gather_dict = joblib.load(open('gather_dict', 'rb'))

In [6]:
thing = stack_dict['img_12.stack.segy']
print(len(thing[0]))

print(len(thing))

# there 1058 instances of 400 elements arrays

400
1058


In [9]:
# turn dictionary into pixel arrays (1058 x 400)
stack_arr = {}

for key in stack_dict.keys():
    arr = stack_dict[key]
    arr = arr.transpose()
    min_arr = arr.min()
    max_arr = arr.max()
    arr = (arr - min_arr) / (max_arr - min_arr)
    arr = arr * 255
    stack_arr[key] = arr


In [10]:
def mode_func(x):
    values, counts = np.unique(x, return_counts=True)
    m = counts.argmax()
    return values[m], 

def kl_divergence(p, q):
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

In [76]:
scaler = MinMaxScaler()

cnt = Counter()

def uncertainty_image(pixel_arr):
    thing = []
    kl_arr = np.zeros((1058, 400, 58))
    i = 0
    for key in pixel_arr.keys():
        j = 0
        for arr in pixel_arr[key]:
            bins = np.linspace(arr.min(), arr.max(), num=10)
            binned_arr = np.digitize(arr, bins)
            
            cnt = Counter()
            for val in binned_arr:
                cnt[val] += 1
            cnt = sorted(cnt.items())
            
            freq_arr = [x[1] for x in cnt]
            max_val = max(freq_arr)
            mode_bin = freq_arr.index(max(freq_arr)) + 1
            onehot_arr = [1e-9 if x != max_val else 1 for x in freq_arr]
            freq_arr = np.array(freq_arr) / 400
            
#             onehot_binarr = np.array([1 if x == mode_val else 1e-9 for x in binned_arr])
#             valhot_binarr = np.array([btwn_val if x == mode_val else 1e-9 for x in binned_arr])

            kl_score = kl_divergence(freq_arr, onehot_arr)
            kl_arr[:, j, i] = kl_score
            j += 1
        max_val = abs(kl_arr[:,:, i].max())
        kl_arr[i,:] = np.array([x / max_val * 256  for x in kl_arr[i,:]])
        i += 1
        
#         max_val = abs(kl_arr.max())
#         kl_arr = np.array([x / max_val * 256  for x in kl_arr])
    for arr in kl_arr:
        thing.append(arr.max())
    im = Image.fromarray(kl_arr)
    im = im.convert('RGB')
    return thing

In [77]:
uncertainty_image(stack_arr)

TypeError: Cannot handle this data type: (1, 1, 58), <f8

In [43]:
arr = stack_dict['img_8.stack.segy'][1057]
bins = np.linspace(arr.min(), arr.max(), num=10)
binned_arr = np.digitize(arr, bins)
binned_arr

array([ 6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  5,  5,  6,  6,  6,  6,  6,
        5,  5,  5,  5,  6,  6,  6,  6,  6,  5,  5,  5,  5,  6,  7,  8,  8,
        6,  3,  2,  3,  6,  9,  9,  6,  4,  3,  5,  7,  7,  7,  5,  3,  3,
        5,  7,  9,  8,  6,  4,  4,  5,  6,  5,  5,  5,  6,  7,  8,  8,  5,
        3,  3,  4,  7,  8,  7,  5,  5,  5,  7,  7,  5,  3,  3,  6,  9,  8,
        5,  3,  4,  6,  7,  6,  5,  5,  6,  6,  6,  6,  6,  5,  4,  4,  6,
        8,  8,  5,  4,  4,  6,  6,  6,  6,  6,  6,  6,  5,  5,  5,  6,  6,
        6,  6,  6,  5,  5,  5,  6,  7,  6,  4,  4,  6,  8, 10,  8,  4,  2,
        2,  4,  7,  8,  8,  9,  8,  6,  2,  1,  2,  5,  8,  9,  7,  6,  5,
        5,  5,  5,  5,  6,  6,  6,  6,  5,  5,  6,  6,  6,  6,  5,  5,  5,
        5,  6,  6,  6,  6,  6,  5,  6,  6,  6,  6,  6,  5,  5,  5,  5,  6,
        6,  7,  6,  6,  5

In [44]:
cnt = Counter()
for val in binned_arr:
    cnt[val] += 1
cnt = sorted(cnt.items())
freq_arr = [x[1] for x in cnt]
max_val = max(freq_arr)
#             print(max_val)
mode_bin = freq_arr.index(max(freq_arr)) + 1
# freq_arr
onehot_arr = [1e-9 if x != max_val else 1 for x in freq_arr]
#             print(onehot_arr)
freq_arr = np.array(freq_arr) / 400

In [45]:
kl_divergence(freq_arr, onehot_arr)

8.8702112348004

In [67]:
kl_arr = np.zeros((1, 400))
i = 0
file = stack_dict['img_8.stack.segy']
for arr in file:
    bins = np.linspace(arr.min(), arr.max(), num=10)
    binned_arr = np.digitize(arr, bins)

    cnt = Counter()
    for val in binned_arr:
        cnt[val] += 1
    cnt = sorted(cnt.items())

    freq_arr = [x[1] for x in cnt]
    max_val = max(freq_arr)
    mode_bin = freq_arr.index(max(freq_arr)) + 1
    onehot_arr = [1e-9 if x != max_val else 1 for x in freq_arr]
    freq_arr = np.array(freq_arr) / 400

#             onehot_binarr = np.array([1 if x == mode_val else 1e-9 for x in binned_arr])
#             valhot_binarr = np.array([btwn_val if x == mode_val else 1e-9 for x in binned_arr])

    kl_score = kl_divergence(freq_arr, onehot_arr)
    kl_arr[0, i] = kl_score
# max_val = abs(kl_arr[i,:].max())
# kl_arr[i,:] = np.array([x / max_val * 256  for x in kl_arr[i,:]])
    i += 1

#         max_val = abs(kl_arr.max())
#         kl_arr = np.array([x / max_val * 256  for x in kl_arr])
for arr in kl_arr:
    thing.append(arr.mean())
im = Image.fromarray(kl_arr)
im = im.convert('RGB')
im

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


IndexError: index 400 is out of bounds for axis 1 with size 400

In [70]:
len(stack_arr.keys())

58